In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
import pandas as pd

In [2]:
sub = pd.read_csv('sample_submission.csv')
print(sub.head())

test = pd.read_csv('test.csv')
test.head()

   ImageId  Label
0        1      0
1        2      0
2        3      0
3        4      0
4        5      0


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# predict

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model = CNN().cuda()
model.load_state_dict(torch.load('mnist_cnn.pt'))

In [4]:
import numpy as np
import torchvision.transforms.functional as vF

transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

sub = {'ImageId':[], 'Label':[]}
# iterate over test dataset
for i, data in test.iterrows():
    x = np.array(data)
    x = torch.from_numpy(x).reshape(1, 28, 28).float()
    x = transform(vF.to_pil_image(x)).reshape(1, 1, 28, 28)
    y = model(x.cuda())
    y = y.max(1, keepdim=True)[1] # get the index of the max log-probability
    sub['ImageId'].append(i+1)
    sub['Label'].append(y.item())
    
pd.DataFrame(sub).to_csv('submission.csv', index=False)

In [6]:
sub_df = pd.read_csv('submission.csv')
print(len(sub_df))
sub_df.head()

28000


,ImageId,Label
0,1,2
1,2,0
2,3,8
3,4,8
4,5,8
